In [2]:
import pandas as pd

In [4]:
from pathlib import Path

In [3]:
from concurrent.futures import ProcessPoolExecutor

In [17]:
project_dir = Path('.').resolve().parents[0]

In [39]:
data_dir = project_dir/'data'/'raw'

In [43]:
def read_csv(path):
    try:
        df = pd.read_csv(path, sep = '\t')
        df['parse_date'] = path.name.rstrip('.tsv')
        return df
    except:
        return None

In [44]:
with ProcessPoolExecutor() as pool:
    dfs = pool.map(read_csv, data_dir.iterdir())

In [45]:
df = pd.concat(dfs)

In [47]:
df.head(1)

,provider,provider_id,name,price,original_price,hidden_price,URL,parse_date
0,bol,9200000093063167,Philips Sonicare ProtectiveClean 4500 HX6830/4...,79.99,NaN,79.99,https://bol.com/nl/nl/p/philips-sonicare-prote...,2021-10-06 09:31AM


In [48]:
df['parse_date'] = pd.to_datetime(df['parse_date'])

In [56]:
df.set_index('parse_date', inplace = True)

In [57]:
df.sort_index(inplace = True)

In [58]:
df[df['name'].str.contains('Little Dutch Memorybox')]

,provider,provider_id,name,price,original_price,hidden_price,URL
parse_date,,,,,,,
2021-10-05 21:01:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-05 21:44:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-05 22:01:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-05 23:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-06 00:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-06 01:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-06 02:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-06 03:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
2021-10-06 04:02:00,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,NaN,https://bol.com/nl/nl/p/little-dutch-memorybox...
